In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from fuzzywuzzy import fuzz
import pandas as pd

left_df = pd.read_csv('/Users/yvonneyue/Downloads/left_dataset.csv')
right_df = pd.read_csv('/Users/yvonneyue/Downloads/right_dataset.csv')

In [3]:
left_df['full_address'] = left_df.apply(lambda row: f"{row['address']}, {row['postal_code']}", axis=1)
left_df.head()

entity_id                      name  \
0          1             The UPS Store   
1          2        St Honore Pastries   
2          3  Perkiomen Valley Brewery   
3          4            Sonic Drive-In   
4          5           Famous Footwear   

                                      address          city state  \
0             87 Grasso Plaza Shopping Center        Affton    MO   
1                                 935 Race St  Philadelphia    PA   
2                               101 Walnut St    Green Lane    PA   
3                               615 S Main St  Ashland City    TN   
4  8522 Eager Road, Dierbergs Brentwood Point     Brentwood    MO   

   postal_code                                         categories  \
0      63123.0  Shipping Centers, Local Services, Notaries, Ma...   
1      19107.0  Restaurants, Food, Bubble Tea, Coffee & Tea, B...   
2      18054.0                          Brewpubs, Breweries, Food   
3      37015.0  Burgers, Fast Food, Sandwiches, Food, Ice Crea...   
4      63144.0  Sporting Goods, Fashion, Shoe Stores, Shopping...   

                                        full_address  
0           87 Grasso Plaza Shopping Center, 63123.0  
1                               935 Race St, 19107.0  
2                             101 Walnut St, 18054.0  
3                             615 S Main St, 37015.0  
4  8522 Eager Road, Dierbergs Brentwood Point, 63...

In [4]:
right_df['simple_zip_code'] = right_df['zip_code'].str.slice(0, 5)
right_df['simple_zip_code'] = right_df['simple_zip_code'].astype(int)
right_df['full_address'] = right_df.apply(lambda row: f"{row['address']}, {row['zip_code']}", axis=1)
right_df.head()

business_id                               name  \
0            1              SOURINI PAINTING INC.   
1            2               WOLFF DOLLA BILL LLC   
2            3  COMPREHENSIVE SURGERY CENTER, LLC   
3            4           FRANK & ADAM APPAREL LLC   
4            5          MORENO PLUS TRANSPORT INC   

                        address        city state    zip_code  size  \
0               12800 44th St N  Clearwater    FL  33762-4726  11.0   
1               1905 E 19th Ave       Tampa    FL  33605-2700   8.0   
2  1988 GULF TO BAY BLVD, Ste 1  CLEARWATER    FL  33765-3550   8.0   
3              13640 Wright Cir       Tampa    FL  33626-3030  12.0   
4     8608 Huron Court unite 58       Tampa    FL       33614   8.0   

   simple_zip_code                              full_address  
0            33762               12800 44th St N, 33762-4726  
1            33605               1905 E 19th Ave, 33605-2700  
2            33765  1988 GULF TO BAY BLVD, Ste 1, 33765-3550  
3            33626              13640 Wright Cir, 33626-3030  
4            33614          8608 Huron Court unite 58, 33614

In [5]:
def calculate_similarity(left, right):
    left = str(left['name']) + str(left['address'])
    right = str(right['name']) + str(right['address'])
    confidence_score = fuzz.token_sort_ratio(left, right)
    return confidence_score

### postal_code and zip_code in -19000

In [6]:
left_df_19000 = left_df[left_df['postal_code'] <= 19000]
right_df_19000 = right_df[right_df['simple_zip_code'] <= 19000]

In [7]:
len(left_df_19000)

2903

In [8]:
matches_19000 = []
for index_left, left in left_df_19000.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19000.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19000.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [9]:
for match in matches_19000:
    match['confidence_score'] /= 100.0

In [10]:
#matches_df_FL = pd.DataFrame(matches_FL)
flt_matches_19000 = [match for match in matches_19000 if match['confidence_score'] > 0.8]
flt_matches_19000[:5]

[{'left_dataset': 394, 'right_dataset': 74086, 'confidence_score': 1.0},
 {'left_dataset': 447, 'right_dataset': 63675, 'confidence_score': 0.93},
 {'left_dataset': 560, 'right_dataset': 61036, 'confidence_score': 0.83},
 {'left_dataset': 787, 'right_dataset': 56561, 'confidence_score': 0.89},
 {'left_dataset': 1003, 'right_dataset': 68036, 'confidence_score': 0.82}]

In [11]:
len(flt_matches_19000)

106

### postal_code and zip_code in 19001-19100

In [12]:
left_df_19001_19100 = left_df[(left_df['postal_code'] >= 19001) & (left_df['postal_code'] <= 19100)]
right_df_19001_19100 = right_df[(right_df['simple_zip_code'] >= 19001) & (right_df['simple_zip_code'] <= 19100)]

In [13]:
len(left_df_19001_19100)

8956

In [14]:
matches_19001_19100 = []
for index_left, left in left_df_19001_19100.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19001_19100.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19001_19100.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [15]:
for match in matches_19001_19100:
    match['confidence_score'] /= 100.0

In [16]:
flt_matches_19001_19100 = [match for match in matches_19001_19100 if match['confidence_score'] > 0.8]
flt_matches_19001_19100[:5]

[{'left_dataset': 67, 'right_dataset': 77383, 'confidence_score': 0.96},
 {'left_dataset': 137, 'right_dataset': 61865, 'confidence_score': 0.83},
 {'left_dataset': 192, 'right_dataset': 65442, 'confidence_score': 0.96},
 {'left_dataset': 572, 'right_dataset': 62533, 'confidence_score': 0.84},
 {'left_dataset': 608, 'right_dataset': 54989, 'confidence_score': 1.0}]

In [17]:
len(flt_matches_19001_19100)

281

### postal_code and zip_code in 19101-19140

In [18]:
left_df_19101_19140 = left_df[(left_df['postal_code'] >= 19100) & (left_df['postal_code'] <= 19140)]
right_df_19101_19140 = right_df[(right_df['simple_zip_code'] >= 19100) & (right_df['simple_zip_code'] <= 19140)]

In [19]:
len(right_df_19101_19140)

10335

In [20]:
matches_19101_19140 = []
for index_left, left in left_df_19101_19140.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19101_19140.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19101_19140.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [21]:
for match in matches_19101_19140:
    match['confidence_score'] /= 100.0

In [22]:
flt_matches_19101_19140 = [match for match in matches_19101_19140 if match['confidence_score'] > 0.8]
flt_matches_19101_19140[:5]

[{'left_dataset': 79, 'right_dataset': 62397, 'confidence_score': 0.89},
 {'left_dataset': 179, 'right_dataset': 69882, 'confidence_score': 0.85},
 {'left_dataset': 507, 'right_dataset': 79724, 'confidence_score': 0.86},
 {'left_dataset': 579, 'right_dataset': 55799, 'confidence_score': 0.87},
 {'left_dataset': 629, 'right_dataset': 65119, 'confidence_score': 0.81}]

In [23]:
len(flt_matches_19101_19140)

648

### postal_code and zip_code in 19141-19410

In [24]:
left_df_19141_19410 = left_df[(left_df['postal_code'] >= 19141) & (left_df['postal_code'] <= 19410)]
right_df_19141_19410 = right_df[(right_df['simple_zip_code'] >= 19141) & (right_df['simple_zip_code'] <= 19410)]

In [25]:
len(right_df_19141_19410)

8219

In [26]:
matches_19141_19410 = []
for index_left, left in left_df_19141_19410.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19141_19410.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19141_19410.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [27]:
for match in matches_19141_19410:
    match['confidence_score'] /= 100.0

In [28]:
flt_matches_19141_19410 = [match for match in matches_19141_19410 if match['confidence_score'] > 0.8]
flt_matches_19141_19410[:5]

[{'left_dataset': 559, 'right_dataset': 55294, 'confidence_score': 0.85},
 {'left_dataset': 715, 'right_dataset': 74025, 'confidence_score': 1.0},
 {'left_dataset': 882, 'right_dataset': 75789, 'confidence_score': 0.86},
 {'left_dataset': 944, 'right_dataset': 78757, 'confidence_score': 0.97},
 {'left_dataset': 1433, 'right_dataset': 58230, 'confidence_score': 1.0}]

In [29]:
len(flt_matches_19141_19410)

462

### postal_code and zip_code in 19411-33600

In [30]:
left_df_19411_33600 = left_df[(left_df['postal_code'] >= 19411) & (left_df['postal_code'] <= 33600)]
right_df_19411_33600 = right_df[(right_df['simple_zip_code'] >= 19411) & (right_df['simple_zip_code'] <= 33600)]

In [31]:
len(right_df_19411_33600)

5558

In [32]:
matches_19411_33600 = []
for index_left, left in left_df_19411_33600.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_19411_33600.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_19411_33600.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [33]:
for match in matches_19411_33600:
    match['confidence_score'] /= 100.0

In [34]:
flt_matches_19411_33600 = [match for match in matches_19411_33600 if match['confidence_score'] > 0.8]
flt_matches_19411_33600[:5]

[{'left_dataset': 159, 'right_dataset': 16722, 'confidence_score': 1.0},
 {'left_dataset': 422, 'right_dataset': 21136, 'confidence_score': 0.83},
 {'left_dataset': 555, 'right_dataset': 69048, 'confidence_score': 1.0},
 {'left_dataset': 1929, 'right_dataset': 73252, 'confidence_score': 0.87},
 {'left_dataset': 1950, 'right_dataset': 69361, 'confidence_score': 0.89}]

In [35]:
len(flt_matches_19411_33600)

275

### postal_code and zip_code in 33601-33640

In [36]:
left_df_33601_33640 = left_df[(left_df['postal_code'] >= 33601) & (left_df['postal_code'] <= 33640)]
right_df_33601_33640 = right_df[(right_df['simple_zip_code'] >= 33601) & (right_df['simple_zip_code'] <= 33640)]

In [37]:
len(right_df_33601_33640)

12935

In [38]:
matches_33601_33640 = []
for index_left, left in left_df_33601_33640.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_33601_33640.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_33601_33640.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [39]:
for match in matches_33601_33640:
    match['confidence_score'] /= 100.0

In [40]:
flt_matches_33601_33640 = [match for match in matches_33601_33640 if match['confidence_score'] > 0.8]
flt_matches_33601_33640[:5]

[{'left_dataset': 195, 'right_dataset': 22389, 'confidence_score': 0.86},
 {'left_dataset': 296, 'right_dataset': 14144, 'confidence_score': 0.89},
 {'left_dataset': 494, 'right_dataset': 8681, 'confidence_score': 0.83},
 {'left_dataset': 705, 'right_dataset': 11573, 'confidence_score': 0.92},
 {'left_dataset': 1254, 'right_dataset': 2768, 'confidence_score': 0.92}]

In [41]:
len(flt_matches_33601_33640)

564

### postal_code and zip_code in 33641-34000

In [42]:
left_df_33641_34000 = left_df[(left_df['postal_code'] >= 33641) & (left_df['postal_code'] <= 34000)]
right_df_33641_34000 = right_df[(right_df['simple_zip_code'] >= 33641) & (right_df['simple_zip_code'] <= 34000)]

In [43]:
len(right_df_33641_34000)

8925

In [44]:
matches_33641_34000 = []
for index_left, left in left_df_33641_34000.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_33641_34000.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_33641_34000.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [45]:
for match in matches_33641_34000:
    match['confidence_score'] /= 100.0

In [46]:
flt_matches_33641_34000 = [match for match in matches_33641_34000 if match['confidence_score'] > 0.8]
flt_matches_33641_34000[:5]

[{'left_dataset': 219, 'right_dataset': 3904, 'confidence_score': 0.85},
 {'left_dataset': 535, 'right_dataset': 16418, 'confidence_score': 0.84},
 {'left_dataset': 942, 'right_dataset': 19690, 'confidence_score': 0.88},
 {'left_dataset': 1106, 'right_dataset': 9700, 'confidence_score': 0.84},
 {'left_dataset': 1287, 'right_dataset': 20985, 'confidence_score': 0.81}]

In [47]:
len(flt_matches_33641_34000)

407

### postal_code and zip_code in 34001-37190

In [48]:
left_df_34001_37190 = left_df[(left_df['postal_code'] >= 34001) & (left_df['postal_code'] <= 37190)]
right_df_34001_37190 = right_df[(right_df['simple_zip_code'] >= 34001) & (right_df['simple_zip_code'] <= 37190)]

In [49]:
len(right_df_34001_37190)

4746

In [50]:
matches_34001_37190 = []
for index_left, left in left_df_34001_37190.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_34001_37190.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_34001_37190.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [51]:
for match in matches_34001_37190:
    match['confidence_score'] /= 100.0

In [52]:
flt_matches_34001_37190 = [match for match in matches_34001_37190 if match['confidence_score'] > 0.8]
flt_matches_34001_37190[:5]

[{'left_dataset': 205, 'right_dataset': 81792, 'confidence_score': 0.84},
 {'left_dataset': 332, 'right_dataset': 89912, 'confidence_score': 0.94},
 {'left_dataset': 1578, 'right_dataset': 87920, 'confidence_score': 1.0},
 {'left_dataset': 1636, 'right_dataset': 80938, 'confidence_score': 0.95},
 {'left_dataset': 1821, 'right_dataset': 84348, 'confidence_score': 0.81}]

In [53]:
len(flt_matches_34001_37190)

182

### postal_code and zip_code in 37191-46190

In [54]:
left_df_37191_46190 = left_df[(left_df['postal_code'] >= 37191) & (left_df['postal_code'] <= 46190)]
right_df_37191_46190 = right_df[(right_df['simple_zip_code'] >= 37191) & (right_df['simple_zip_code'] <= 46190)]

In [55]:
len(right_df_37191_46190)

8618

In [56]:
matches_37191_46190 = []
for index_left, left in left_df_37191_46190.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_37191_46190.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_37191_46190.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [57]:
for match in matches_37191_46190:
    match['confidence_score'] /= 100.0

In [58]:
flt_matches_37191_46190 = [match for match in matches_37191_46190 if match['confidence_score'] > 0.8]
flt_matches_37191_46190[:5]

[{'left_dataset': 540, 'right_dataset': 88547, 'confidence_score': 0.95},
 {'left_dataset': 696, 'right_dataset': 87455, 'confidence_score': 0.86},
 {'left_dataset': 1467, 'right_dataset': 85001, 'confidence_score': 0.81},
 {'left_dataset': 1483, 'right_dataset': 90965, 'confidence_score': 0.91},
 {'left_dataset': 1493, 'right_dataset': 81315, 'confidence_score': 0.84}]

In [59]:
len(flt_matches_37191_46190)

454

### postal_code and zip_code in 46191-63090

In [60]:
left_df_46191_63090 = left_df[(left_df['postal_code'] >= 46191) & (left_df['postal_code'] <= 63090)]
right_df_46191_63090 = right_df[(right_df['simple_zip_code'] >= 46191) & (right_df['simple_zip_code'] <= 63090)]

In [61]:
len(right_df_46191_63090)

11779

In [62]:
matches_46191_63090 = []
for index_left, left in left_df_46191_63090.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_46191_63090.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_46191_63090.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [63]:
for match in matches_46191_63090:
    match['confidence_score'] /= 100.0

In [64]:
flt_matches_46191_63090 = [match for match in matches_46191_63090 if match['confidence_score'] > 0.8]
flt_matches_46191_63090[:5]

[{'left_dataset': 57, 'right_dataset': 32737, 'confidence_score': 0.81},
 {'left_dataset': 411, 'right_dataset': 31981, 'confidence_score': 0.84},
 {'left_dataset': 622, 'right_dataset': 34070, 'confidence_score': 0.86},
 {'left_dataset': 1338, 'right_dataset': 33842, 'confidence_score': 0.86},
 {'left_dataset': 1344, 'right_dataset': 32692, 'confidence_score': 0.81}]

In [65]:
len(flt_matches_46191_63090)

410

### postal_code and zip_code in 63091-

In [66]:
left_df_63091 = left_df[left_df['postal_code'] >= 63091]
right_df_63091 = right_df[right_df['simple_zip_code'] >= 63091]

In [67]:
len(right_df_63091)

12929

In [68]:
matches_63091 = []
for index_left, left in left_df_63091.iterrows():
    best_match = None
    best_score = 0
    for index_right, right in right_df_63091.iterrows():
        score = calculate_similarity(left, right)
        if score > best_score:
            best_score = score
            best_match = right
    matches_63091.append({'left_dataset': left['entity_id'], 'right_dataset': best_match['business_id'], 'confidence_score': best_score})

In [69]:
for match in matches_63091:
    match['confidence_score'] /= 100.0

In [70]:
flt_matches_63091 = [match for match in matches_63091 if match['confidence_score'] > 0.8]
flt_matches_63091[:5]

[{'left_dataset': 60, 'right_dataset': 39159, 'confidence_score': 0.83},
 {'left_dataset': 139, 'right_dataset': 51282, 'confidence_score': 0.88},
 {'left_dataset': 378, 'right_dataset': 37129, 'confidence_score': 0.87},
 {'left_dataset': 662, 'right_dataset': 48146, 'confidence_score': 0.82},
 {'left_dataset': 759, 'right_dataset': 36677, 'confidence_score': 0.82}]

In [71]:
len(flt_matches_63091)

491

In [ ]:
# Sum all matching rows

In [72]:
len(flt_matches_19000) + len(flt_matches_19001_19100) + len(flt_matches_19101_19140) + len(flt_matches_19141_19410) + len(flt_matches_19411_33600) + len(flt_matches_33601_33640) + len(flt_matches_33641_34000) + len(flt_matches_34001_37190) + len(flt_matches_37191_46190) + len(flt_matches_46191_63090) + len(flt_matches_63091)

4280

In [85]:
result = pd.concat([pd.DataFrame(flt_matches_19000), 
                    pd.DataFrame(flt_matches_19001_19100), 
                    pd.DataFrame(flt_matches_19101_19140),
                    pd.DataFrame(flt_matches_19141_19410),
                    pd.DataFrame(flt_matches_19411_33600),
                    pd.DataFrame(flt_matches_33601_33640),
                    pd.DataFrame(flt_matches_33641_34000),
                    pd.DataFrame(flt_matches_34001_37190),
                    pd.DataFrame(flt_matches_37191_46190),
                    pd.DataFrame(flt_matches_46191_63090),
                    pd.DataFrame(flt_matches_63091)
                   ], ignore_index=True)

In [86]:
result.to_csv('output.csv', index=False)